# Nessie Catalog
Now we test the Nessie catalog.

## Importing Required Libraries
As before, we will be importing `SparkSession` and `os`, which is used to read environment variable for the Minio access key and secret.

We also set some styling to display tables better.

In [1]:
from pyspark.sql import SparkSession
import os

# this is to better display pyspark dataframes
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

## Setting up Spark Session
Details docs of the spark configs to use with the Nessie catalog can be found [here](https://projectnessie.org/guides/spark-s3/#setting-up-spark-session-for-amazon-s3) and [here](https://iceberg.apache.org/docs/1.5.1/nessie/).
We will setting up `iceberg_nessie` as the catalog name.

In [2]:
iceberg_catalog_name = "iceberg_nessie"
spark = SparkSession.builder \
  .appName("iceberg-nessie") \
  .config("spark.driver.memory", "4g") \
  .config("spark.executor.memory", "4g") \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions") \
  .config("spark.jars", "/opt/extra-jars/iceberg-spark-runtime.jar,/opt/extra-jars/iceberg-aws-bundle.jar,/opt/extra-jars/nessie-spark-extensions-3.5_2.13.jar") \
  .config(f"spark.sql.catalog.{iceberg_catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
  .config(f"spark.sql.catalog.{iceberg_catalog_name}.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog") \
  .config(f"spark.sql.catalog.{iceberg_catalog_name}.uri", "http://nessie:19120/api/v1") \
  .config(f"spark.sql.catalog.{iceberg_catalog_name}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
  .config(f"spark.sql.catalog.{iceberg_catalog_name}.warehouse", "s3://warehouse/iceberg-nessie/") \
  .config(f"spark.sql.catalog.{iceberg_catalog_name}.s3.endpoint", "http://minio:9000") \
  .config(f"spark.sql.catalog.{iceberg_catalog_name}.s3.path-style-access", "true") \
  .getOrCreate()


24/09/01 15:58:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Setting up the test parquet file as a dataframe

In [3]:
df = spark.read.parquet("file:///home/iceberg/workspace/downloaded-data/yellow_tripdata_2024-01.parquet")

## Creating Iceberg namespace under the catalog
Now we created the namespace`iceberg_nessie`, with the location `s3://warehouse/iceberg-nessie/` in Minio

In [6]:
spark.sql("create schema if not exists iceberg_nessie.iceberg_nessie")

Py4JJavaError: An error occurred while calling o47.sql.
: java.lang.AbstractMethodError: Missing implementation of resolved method 'abstract scala.collection.Seq apply(org.apache.spark.sql.catalyst.plans.logical.LogicalPlan)' of abstract class org.apache.spark.sql.catalyst.planning.GenericStrategy.
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:93)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:70)
	at org.apache.spark.sql.execution.QueryExecution$.createSparkPlan(QueryExecution.scala:496)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$sparkPlan$1(QueryExecution.scala:171)
	at org.apache.spark.sql.catalyst.QueryPlanningT

In [5]:
df.writeTo("iceberg_nessie.iceberg_nessie.yellow_tripdata_2024_01").create()

Py4JJavaError: An error occurred while calling o51.writeTo.
: java.lang.AbstractMethodError: Receiver class org.apache.spark.sql.catalyst.parser.extensions.NessieSparkSqlExtensionsParser does not define or inherit an implementation of the resolved method 'abstract scala.collection.Seq parseMultipartIdentifier(java.lang.String)' of interface org.apache.spark.sql.catalyst.parser.ParserInterface.
	at org.apache.spark.sql.DataFrameWriterV2.<init>(DataFrameWriterV2.scala:45)
	at org.apache.spark.sql.Dataset.writeTo(Dataset.scala:4021)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:75)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:52)
	at java.base/java.lang.reflect.Method.invoke(Method.java:580)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke

In [12]:
spark.stop()